In [57]:
import sys, os
sys.argv = ['']  # argparse 충돌 방지용
# 프로젝트 루트 경로를 조정하세요
proj_root = os.path.abspath(os.path.join('..'))
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)
print("▶️ 프로젝트 루트:", proj_root)
print("Python executable:", sys.executable)

▶️ 프로젝트 루트: C:\Users\USER\Documents\GitHub\medication_object_detection_project_team2\Project
Python executable: C:\Users\USER\AppData\Local\pypoetry\Cache\virtualenvs\meditation-detection-project-ihVzpsAT-py3.11\Scripts\python.exe


In [58]:
import random
import torch
import matplotlib.pyplot as plt

# ultralytics YOLO와 설정, 매핑, 시각화 유틸
from ultralytics import YOLO
from src.config import get_config, get_device
from src.utils.class_mapping import get_class_mapping
from src.utils.visualizer       import visualize_predictions


In [59]:
from src.config import get_config, get_device

cfg    = get_config()
device = get_device()
print(f"▶ Device: {device}, test_img_dir: {cfg.test_image_dir}")

▶ Device: cuda, test_img_dir: C:\Users\USER\Documents\GitHub\medication_object_detection_project_team2\data\ai03-level1-project\test_images


In [60]:
# visualize_train.py
import json, glob, os
import argparse
from pathlib import Path
import cv2

# 1) config 로드
from src.config import get_config, get_device
cfg = get_config()
device = get_device()

# 2) 경로 세팅
#   - annotation 원본 폴더
ann_root = Path(cfg.annotation_dir)
#   - coco 저장 폴더: 프로젝트 루트 아래 coco/
coco_dir = Path(cfg.output_dir).parent / "coco"
coco_dir.mkdir(exist_ok=True, parents=True)
coco_json = coco_dir / "coco_train.json"
#   - train 이미지 폴더
img_dir = Path(cfg.train_image_dir)
#   - 시각화 출력 폴더
vis_dir = img_dir.parent / "vis_train"
vis_dir.mkdir(exist_ok=True, parents=True)

print(f"▶ Device       : {device}")
print(f"▶ Ann root     : {ann_root}")
print(f"▶ Coco JSON    : {coco_json}")
print(f"▶ Train images : {img_dir}")
print(f"▶ Vis output   : {vis_dir}")

▶ Device       : cuda
▶ Ann root     : C:\Users\USER\Documents\GitHub\medication_object_detection_project_team2\data\ai03-level1-project\train_annotations
▶ Coco JSON    : C:\Users\USER\Documents\GitHub\medication_object_detection_project_team2\Project\coco\coco_train.json
▶ Train images : C:\Users\USER\Documents\GitHub\medication_object_detection_project_team2\data\ai03-level1-project\train_images
▶ Vis output   : C:\Users\USER\Documents\GitHub\medication_object_detection_project_team2\data\ai03-level1-project\vis_train


In [36]:
# 3) COCO 합치기
all_images = []
all_anns   = []
cat_map    = {}    # id → {id,name,supercategory}
ann_id     = 0

# 재귀적으로 모든 JSON 읽기
for jp in ann_root.rglob("*.json"):
    data = json.loads(jp.read_text(encoding="utf-8"))
    # images
    all_images.extend(data.get("images", []))
    # categories
    for cat in data.get("categories", []):
        cid = cat["id"]
        if cid not in cat_map:
            cat_map[cid] = {
                "id": cid,
                "name": cat.get("name",""),
                "supercategory": cat.get("supercategory","")
            }
    # annotations (bbox 유효성 + id 재할당)
    for ann in data.get("annotations", []):
        bbox = ann.get("bbox", [])
        if isinstance(bbox, list) and len(bbox)==4:
            ann_id += 1
            ann["id"] = ann_id
            all_anns.append(ann)

# 중복 없는 images
unique_imgs = {img["id"]: img for img in all_images}

# 최종 coco dict
coco = {
    "images":      list(unique_imgs.values()),
    "annotations": all_anns,
    "categories":  list(cat_map.values())
}

# 저장
with open(coco_json, "w", encoding="utf-8") as f:
    json.dump(coco, f, ensure_ascii=False, indent=2)

print(f"✅ coco_train.json 생성: images={len(coco['images'])}, anns={len(coco['annotations'])}, cats={len(coco['categories'])}")

✅ coco_train.json 생성: images=1489, anns=4526, cats=73


In [48]:
import sys, os, json
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import font_manager
from PIL import Image, ImageDraw, ImageFont

# argparse 충돌 방지
sys.argv = ['']

# 1) config 로드
from src.config import get_config, get_device
cfg    = get_config()
device = get_device()

# 2) 경로 세팅
ann_root = Path(cfg.annotation_dir)                        # 원본 annotations 폴더
img_dir  = Path(cfg.train_image_dir)                       # train_images
vis_dir  = img_dir.parent / "vis_train"                    # 출력 폴더
vis_dir.mkdir(exist_ok=True, parents=True)

coco_dir  = Path(cfg.output_dir).parent / "coco"           # coco json 폴더
coco_json = coco_dir / "coco_train.json"

print("▶ Device       :", device)
print("▶ train images :", img_dir)
print("▶ vis output   :", vis_dir)
print("▶ coco json    :", coco_json)

# 3) 한글 폰트 matplotlib에 등록
FONT_PATH = "C:/Windows/Fonts/malgun.ttf"  # 맑은고딕
font_manager.fontManager.addfont(FONT_PATH)
plt.rcParams["font.family"] = font_manager.FontProperties(fname=FONT_PATH).get_name()
plt.rcParams["axes.unicode_minus"] = False

# 4) COCO JSON 로드 & 매핑
coco = json.loads(coco_json.read_text(encoding="utf-8"))
images_map = {img["id"]: img for img in coco["images"]}
anns_map   = {}
for ann in coco["annotations"]:
    anns_map.setdefault(ann["image_id"], []).append(ann)
cats_map   = {c["id"]: c["name"] for c in coco["categories"]}

# 5) 시각화 함수 (PIL + matplotlib)
def visualize_one(image_info, ann_list):
    img_path = img_dir / image_info["file_name"]
    pil = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil)
    font = ImageFont.truetype(FONT_PATH, size=20)

    for ann in ann_list:
        x, y, w, h = map(int, ann["bbox"])
        label = cats_map.get(ann["category_id"], "")

        # 박스
        draw.rectangle([x, y, x+w, y+h], outline="red", width=2)

        # 텍스트 크기 계산 (textbbox 사용)
        bbox_txt = draw.textbbox((0, 0), label, font=font)
        tw, th = bbox_txt[2] - bbox_txt[0], bbox_txt[3] - bbox_txt[1]

        # 텍스트 배경
        draw.rectangle([x, y-th, x+tw, y], fill="red")
        # 텍스트
        draw.text((x, y-th), label, font=font, fill="white")

    return pil


# 6) 전체 이미지 시각화 (or 샘플만)
for img_id, info in images_map.items():
    ann_list = anns_map.get(img_id, [])
    vis_img = visualize_one(info, ann_list)
    out_path = vis_dir / info["file_name"]
    vis_img.save(out_path)
    # 진행 표시(원하는 빈도로 조정)
    if img_id % 100 == 0:
        print(f"  ▪ {img_id}/{len(images_map)} saved")

print("🎉 전량 시각화 완료, vis_train 폴더를 확인하세요.")


▶ Device       : cuda
▶ train images : C:\Users\USER\Documents\GitHub\medication_object_detection_project_team2\data\ai03-level1-project\train_images
▶ vis output   : C:\Users\USER\Documents\GitHub\medication_object_detection_project_team2\data\ai03-level1-project\vis_train
▶ coco json    : C:\Users\USER\Documents\GitHub\medication_object_detection_project_team2\Project\coco\coco_train.json
  ▪ 100/1489 saved
  ▪ 1000/1489 saved
  ▪ 300/1489 saved
  ▪ 400/1489 saved
  ▪ 600/1489 saved
  ▪ 1500/1489 saved
  ▪ 700/1489 saved
  ▪ 500/1489 saved
  ▪ 800/1489 saved
  ▪ 1200/1489 saved
  ▪ 1400/1489 saved
  ▪ 1300/1489 saved
  ▪ 200/1489 saved
  ▪ 1100/1489 saved
  ▪ 900/1489 saved
🎉 전량 시각화 완료, vis_train 폴더를 확인하세요.


In [61]:
import json
from pathlib import Path

# 1) annotation 원본 폴더
ann_root = Path(cfg.annotation_dir)
ANN_DIR = ann_root

total       = 0
valid_count = 0
invalid     = []

for jp in ANN_DIR.rglob("*.json"):
    data = json.loads(jp.read_text(encoding="utf-8"))
    for ann in data.get("annotations", []):
        total += 1
        bbox = ann.get("bbox", None)
        # 존재 조건: bbox 키가 없거나 None
        if bbox is None:
            invalid.append((jp, ann.get("id"), "missing"))
        # 유효 조건: 리스트 형태 + 길이 4
        elif not (isinstance(bbox, list) and len(bbox) == 4):
            invalid.append((jp, ann.get("id"), f"bad_length={len(bbox) if isinstance(bbox, list) else type(bbox)}"))
        else:
            valid_count += 1

# 2) 결과 출력
print(f"전체 어노테이션 건수      : {total}")
print(f"✅ 유효한 bbox 건수      : {valid_count}")
print(f"❗ 유효하지 않은 bbox 건수: {len(invalid)}")

if invalid:
    print("\n== 유효하지 않은 샘플 (최대 20개) ==")
    for fn, aid, reason in invalid[:20]:
        print(f"  파일: {fn.relative_to(ANN_DIR)}, ann_id={aid}, 이유={reason}")
    if len(invalid) > 20:
        print(f"  ... 그리고 총 {len(invalid)}건")


전체 어노테이션 건수      : 4526
✅ 유효한 bbox 건수      : 4526
❗ 유효하지 않은 bbox 건수: 0


In [62]:

# 파일 단위 카운터
total_files    = 0
files_with_seg = 0

# 어노테이션 단위 카운터
total_anns        = 0
anns_with_seg     = 0
anns_without_seg  = 0

for jp in ANN_DIR.rglob("*.json"):
    total_files += 1
    data = json.loads(jp.read_text(encoding="utf-8"))
    
    # 이 파일에 segmentation이 하나라도 있는지 체크
    file_has_seg = False
    
    for ann in data.get("annotations", []):
        total_anns += 1
        seg = ann.get("segmentation", None)
        
        if isinstance(seg, list) and len(seg) > 0:
            anns_with_seg += 1
            file_has_seg = True
        else:
            anns_without_seg += 1
    
    if file_has_seg:
        files_with_seg += 1

# 파일 단위 출력
print("▶ JSON 파일 전체 개수            :", total_files)
print("▶ segmentation 아이템 있는 파일  :", files_with_seg)
print("▶ segmentation 모두 빈 파일      :", total_files - files_with_seg)

print()

# 어노테이션 단위 출력
print("▶ 전체 어노테이션 개수          :", total_anns)
print("▶ segmentation 아이템 있는 어노테이션 :", anns_with_seg)
print("▶ segmentation 빈 어노테이션        :", anns_without_seg)

▶ JSON 파일 전체 개수            : 4526
▶ segmentation 아이템 있는 파일  : 0
▶ segmentation 모두 빈 파일      : 4526

▶ 전체 어노테이션 개수          : 4526
▶ segmentation 아이템 있는 어노테이션 : 0
▶ segmentation 빈 어노테이션        : 4526


In [63]:
import json
from pathlib import Path
import pandas as pd

# ── 1) JSON 파일 리스트 ──
json_files = list(ANN_DIR.rglob("*.json"))

rows = []

# ── 2) 각 JSON 파일마다 rows에 1개 이상의 dict 쌓기 ──
for jf in json_files:
    data = json.loads(jf.read_text(encoding="utf-8"))

    # 이미지 메타는 보통 하나이므로 첫 번째만
    img = data.get("images", [{}])[0]

    # 카테고리 메타맵 (id→meta)
    cat_map = {c["id"]: c for c in data.get("categories", [])}

    # 최상위 type
    root_type = data.get("type")

    for ann in data.get("annotations", []):
        rec = {}

        # 1) images 섹션의 필드 (약 50개) → 접두어 images_ 로 바꿈
        for k, v in img.items():
            rec["images_" + k] = v
        # 예: images_file_name, images_width, images_height, images_dl_idx, ...

        # 2) 최상위 type
        rec["root_type"] = root_type

        # 3) annotations 섹션의 필드 (8개)
        rec["annotations_area"]      = ann.get("area")
        rec["annotations_iscrowd"]   = ann.get("iscrowd", 0)
        rec["annotations_ignore"]    = ann.get("ignore", 0)
        rec["annotations_bbox"]      = ann.get("bbox")
        # id는 annotations_id
        rec["annotations_id"]        = ann.get("id")
        # image_id → images_id 와 동일
        rec["annotations_image_id"]  = ann.get("image_id")
        # category_id → categories_id
        rec["annotations_category_id"] = ann.get("category_id")
        # segmentation
        rec["annotations_segmentation"] = ann.get("segmentation")

        # bbox 개별 컬럼으로 분리
        bbox = ann.get("bbox", [])
        if isinstance(bbox, list) and len(bbox)==4:
            rec["annotations_bbox_x"], rec["annotations_bbox_y"], rec["annotations_bbox_w"], rec["annotations_bbox_h"] = bbox
        else:
            rec["annotations_bbox_x"] = rec["annotations_bbox_y"] = rec["annotations_bbox_w"] = rec["annotations_bbox_h"] = None

        # 4) categories 메타 (3개)
        cid = ann.get("category_id")
        cat = cat_map.get(cid, {})
        rec["categories_supercategory"] = cat.get("supercategory")
        rec["categories_name"]          = cat.get("name")
        # 카테고리 메타의 id 컬럼도 붙이고 싶으면:
        rec["categories_id"]            = cid

        rows.append(rec)

# ── 3) DataFrame 생성 ──
df = pd.DataFrame(rows)

# 결과 확인
print("Rows:", df.shape[0], "Columns:", df.shape[1])
print(df.columns.tolist())


Rows: 4526 Columns: 66
['images_file_name', 'images_width', 'images_height', 'images_imgfile', 'images_drug_N', 'images_drug_S', 'images_back_color', 'images_drug_dir', 'images_light_color', 'images_camera_la', 'images_camera_lo', 'images_size', 'images_dl_idx', 'images_dl_mapping_code', 'images_dl_name', 'images_dl_name_en', 'images_img_key', 'images_dl_material', 'images_dl_material_en', 'images_dl_custom_shape', 'images_dl_company', 'images_dl_company_en', 'images_di_company_mf', 'images_di_company_mf_en', 'images_item_seq', 'images_di_item_permit_date', 'images_di_class_no', 'images_di_etc_otc_code', 'images_di_edi_code', 'images_chart', 'images_drug_shape', 'images_thick', 'images_leng_long', 'images_leng_short', 'images_print_front', 'images_print_back', 'images_color_class1', 'images_color_class2', 'images_line_front', 'images_line_back', 'images_img_regist_ts', 'images_form_code_name', 'images_mark_code_front_anal', 'images_mark_code_back_anal', 'images_mark_code_front_img', 'i

In [64]:
# 세 컬럼이 모두 같은 행 필터링
mask = (
    df['images_dl_idx'].astype(str) == df['annotations_category_id'].astype(str)
) & (
    df['images_dl_idx'].astype(str) == df['categories_id'].astype(str)
)
matching_count = mask.sum()

print(f"세 컬럼(images_dl_idx, annotations_category_id, categories_id)이 모두 일치하는 행의 개수: {matching_count}")


세 컬럼(images_dl_idx, annotations_category_id, categories_id)이 모두 일치하는 행의 개수: 4526


In [65]:
# 1) 세 컬럼 타입 통일 (string)
df['images_dl_idx_str']           = df['images_dl_idx'].astype(str)
df['annotations_category_id_str'] = df['annotations_category_id'].astype(str)
df['categories_id_str']           = df['categories_id'].astype(str)

# 2) 필터링: images_id == annotations_image_id
mask1 = df['images_id'] == df['annotations_image_id']

# 3) 필터링: images_dl_idx == annotations_category_id == categories_id
mask2 = (
    (df['images_dl_idx_str'] == df['annotations_category_id_str']) &
    (df['images_dl_idx_str'] == df['categories_id_str'])
)

# 4) 두 조건을 모두 만족하는 행의 개수 계산
matching_count = (mask1 & mask2).sum()
print(f"조건을 모두 만족하는 행의 개수: {matching_count}")


조건을 모두 만족하는 행의 개수: 4526


In [66]:
unique_count = df['images_id'].nunique()
print(f"images_id의 고유값 개수: {unique_count}")


images_id의 고유값 개수: 1489


In [68]:
import pandas as pd

# 1) images_id당 어노테이션(행) 개수 세기
counts = df['images_id'].value_counts()

# 2) “어노테이션 개수”별로 “images_id 수” 집계
dup_summary = counts.value_counts().sort_index()

# 3) 결과 출력
print("어노테이션 개수  |  images_id의 개수")
print("----------------+-------------------")
for ann_cnt, img_cnt in dup_summary.items():
    print(f"{ann_cnt:14d} | {img_cnt:17d}")


어노테이션 개수  |  images_id의 개수
----------------+-------------------
             1 |                64
             2 |               302
             3 |               634
             4 |               489


In [70]:
import json
from pathlib import Path
from collections import defaultdict
import pandas as pd

# ── 1) 원본 JSON들을 순회하며 정보 수집 ──
img_file_map  = {}              # image_id → file_name
pill_count    = {}              # image_id → 실제 pill 개수
ann_count     = defaultdict(int)  # image_id → 어노테이션 누적 개수

for jf in ANN_DIR.rglob("*.json"):
    data = json.loads(jf.read_text(encoding="utf-8"))
    
    # 1-1) images 항목에서 file_name, id 저장
    for img in data.get("images", []):
        iid = img["id"]
        fn  = img["file_name"]
        img_file_map[iid] = fn
        
        # file_name에서 '_' 앞부분을 잘라 '-' 개수 세기 → pill_count
        # (첫 토큰 'K' 제외)
        base = fn.split("_", 1)[0]
        cnt  = max(0, len(base.split("-")) - 1)
        pill_count[iid] = cnt
    
    # 1-2) annotations 항목을 모두 세기
    for ann in data.get("annotations", []):
        iid = ann["image_id"]
        ann_count[iid] += 1

# ── 2) DataFrame 생성 ──
rows = []
for iid, fn in img_file_map.items():
    rows.append({
        "image_id"   : iid,
        "file_name"  : fn,
        "pill_count" : pill_count.get(iid, 0),
        "anno_count" : ann_count.get(iid, 0),
        "match"      : pill_count.get(iid, 0) == ann_count.get(iid, 0)
    })
df = pd.DataFrame(rows)

# ── 3) 결과 확인 ──
print("전체 이미지 수:", len(df))
print("불일치 케이스 수:", df.query("match==False").shape[0])
print("\n불일치 상위 10개 샘플:")
print(df.query("match==False").sort_values(
    ["pill_count","anno_count"], ascending=False
).head(10))

print("\npill_count × anno_count 분포표:")
print(df.pivot_table(
    index="pill_count",
    columns="anno_count",
    values="image_id",
    aggfunc="count",
    fill_value=0
))

전체 이미지 수: 1489
불일치 케이스 수: 850

불일치 상위 10개 샘플:
    image_id                                          file_name  pill_count  \
0       1417  K-001900-010224-016551-031705_0_2_0_2_70_000_2...           4   
2       1419  K-001900-010224-016551-031705_0_2_0_2_90_000_2...           4   
5        410  K-001900-010224-016551-033009_0_2_0_2_90_000_2...           4   
9       1270  K-001900-016548-018110-027926_0_2_0_2_70_000_2...           4   
11      1272  K-001900-016548-018110-027926_0_2_0_2_90_000_2...           4   
13      1290  K-001900-016548-018110-029345_0_2_0_2_75_000_2...           4   
20       966  K-001900-016548-018110-031705_0_2_0_2_70_000_2...           4   
25       494  K-001900-016548-019607-021026_0_2_0_2_75_000_2...           4   
36       740  K-001900-016548-019607-031705_0_2_0_2_70_000_2...           4   
43      1069  K-001900-016548-021026-021771_0_2_0_2_90_000_2...           4   

    anno_count  match  
0            3  False  
2            3  False  
5           